In [1]:
!wget -nc https://github.com/lazyprogrammer/machine_learning_examples/blob/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://github.com/lazyprogrammer/machine_learning_examples/blob/master/hmm_class/robert_frost.txt


--2024-04-11 04:15:46--  https://github.com/lazyprogrammer/machine_learning_examples/blob/master/hmm_class/edgar_allan_poe.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt     [ <=>                ] 182.57K  --.-KB/s    in 0.01s   

2024-04-11 04:15:46 (15.1 MB/s) - ‘edgar_allan_poe.txt’ saved [186955]

--2024-04-11 04:15:46--  https://github.com/lazyprogrammer/machine_learning_examples/blob/master/hmm_class/robert_frost.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘robert_frost.txt’

robert_frost.txt        [ <=>                ] 215.33K  --.-KB/s    in 0.009s  

2024-04-11 04:15:47 (22.4 MB/s) - ‘robert_frost.txt’

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [8]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt',
]

In [9]:
!head robert_frost.txt


Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [10]:
input_texts = []
labels= []

for label,f in enumerate(input_files):
  for line in open(f):
    line = line.rstrip().lstrip()
    if line:
      line = line.translate(str.maketrans('','',string.punctuation))
      input_texts.append(line)
      labels.append(label)

In [11]:
train_text , test_text ,Ytrain,Ytest = train_test_split(input_texts,labels)

In [12]:
len(Ytrain), len(Ytest)


(1615, 539)

In [13]:
idx = 1
word2idx = { '<unk>' : 0 }

for text in train_text:
  tokens  = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx +=1

In [14]:
word2idx

{'<unk>': 0,
 'Take': 1,
 'it': 2,
 'year': 3,
 'in': 4,
 'out': 5,
 'he': 6,
 'doesnt': 7,
 'make': 8,
 'much': 9,
 'An': 10,
 'hour': 11,
 'of': 12,
 'winter': 13,
 'day': 14,
 'might': 15,
 'seem': 16,
 'too': 17,
 'short': 18,
 'A': 19,
 'passionate': 20,
 'light': 21,
 'such': 22,
 'for': 23,
 'his': 24,
 'spirit': 25,
 'was': 26,
 'fit': 27,
 'Roses': 28,
 'that': 29,
 'grew': 30,
 'an': 31,
 'enchanted': 32,
 'garden': 33,
 'In': 34,
 'every': 35,
 'way': 36,
 'you': 37,
 'may': 38,
 'please': 39,
 'yourself': 40,
 'She': 41,
 'wont': 42,
 'get': 43,
 'far': 44,
 'before': 45,
 'the': 46,
 'wheels': 47,
 'hang': 48,
 'up': 49,
 'Save': 50,
 'but': 51,
 'soul': 52,
 'thine': 53,
 'uplifted': 54,
 'eyes': 55,
 'Only': 56,
 'well': 57,
 'I': 58,
 'knew': 59,
 'there': 60,
 'must': 61,
 'be': 62,
 'something': 63,
 'rolls': 64,
 'through': 65,
 'ether': 66,
 'sighs': 67,
 'turned': 68,
 'and': 69,
 'repented': 70,
 'coming': 71,
 'back': 72,
 'The': 73,
 'poetess': 74,
 'who': 75,
 

In [16]:
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int =[word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token,0) for token in tokens]
  test_text_int.append(line_as_int)


In [17]:
v = len(word2idx)

A0 = np.ones((v,v))
pi0 = np.ones(v)

A1 = np.ones((v,v))
pi1 = np.ones(v)

In [20]:
def compute_counts(text_as_int,A,pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        pi[idx] += 1
      else:
        A[last_idx,idx] +=1
      last_idx = idx


In [24]:
compute_counts([t for t,y in zip(train_text_int,Ytrain) if y==0],A0,pi0)
compute_counts([t for t,y in zip(test_text_int,Ytest) if y==1],A1,pi0)

In [25]:
A0 /= A0.sum(axis=1,keepdims=True)
pi0 /=pi0.sum()

A1 /= A1.sum(axis=1,keepdims = True)
pi1 /= pi1.sum()

In [26]:
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [27]:
count0 = sum(y==0 for y in Ytrain)
count1 = sum(y ==1 for y in Ytrain)
total =len(Ytrain)
p0 = count0/total
p1 = count1/total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0,p1

(0.34241486068111454, 0.6575851393188854)

In [52]:
class Classifier:
  def __init__(self,logAs,logpis,logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.k = len(logpriors)

  def _compute_log_likelihood(self,input_,class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx,idx]
      last_idx = idx

    return logprob

  def predict(self,inputs):
      predictions = np.zeros(len(inputs))
      for i ,input_ in enumerate(inputs):
        posteriors = [self._compute_log_likelihood(input_,c) + self.logpriors[c]
                      for c in range(self.k)]
        pred = np.argmax(posteriors)
        predictions[i] = pred
      return predictions

In [53]:
clf = Classifier([logA0,logA1],[logpi0,logpi1],[logp0,logp1])

In [63]:
ptrain = clf.predict(train_text_int)
print(f"train acc:{np.mean(ptrain == Ytrain)} ")


train acc:0.6910216718266254 


In [58]:
ptest = clf.predict(test_text_int)
print(f"test acc : {np.mean(ptest == Ytest)}")

test acc : 0.8534322820037106


In [59]:
from sklearn.metrics import confusion_matrix,f1_score


In [64]:
cm = confusion_matrix(Ytrain,ptrain)
cm

array([[553,   0],
       [499, 563]])

In [65]:
cm_test = confusion_matrix(Ytest,ptest)
cm_test

array([[ 91,  74],
       [  5, 369]])

In [66]:
f1_score(Ytrain,ptrain)

0.692923076923077

In [67]:
f1_score(Ytest,ptest)

0.9033047735618116